# Space

In [1]:
import os
import sys 
import logging
import random
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
# pprint(SPACE)

# Available Packages
import argparse
import datasets
import pandas as pd
from datetime import datetime 


from recfldtkn.configfn import load_cohort_args
from recfldtkn.loadtools import load_module_variables, update_args_to_list
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer
from config_observer.CKPD import ckpd_to_CkpdObsConfig

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


g:\Shared drives\CDHAI-WellDoc\2024-WellDocTest-SPACE\_WellDoc-AI-CGMGPT-WorkSpace


# [Part 1] Get a Caseset: Case Examples

## [Step 1]

In [2]:
from recfldtkn.loadtools import fetch_TriggerEvent_tools

####################
TriggerCaseMethod = 'CGM5MinEntry'
# TriggerCaseMethod = 'FoodEntryEOD'
####################

Trigger_tools = fetch_TriggerEvent_tools(TriggerCaseMethod, SPACE)
# Trigger_tools 
case_id_columns = Trigger_tools['case_id_columns']
special_columns = Trigger_tools['special_columns']

In [3]:
##################################
CaseSetName = TriggerCaseMethod
case_id_columns = Trigger_tools['case_id_columns']
special_columns = Trigger_tools['special_columns']
##################################

TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)

cohort_args['ckpd_to_CkpdObsConfig'] = ckpd_to_CkpdObsConfig
cohort_args['case_id_columns'] = case_id_columns
cohort_args['ObsDTName'] = 'ObsDT'
cohort_args['PID_ObsDT_columns'] = [cohort_args['RootID'], cohort_args['ObsDTName']]

print(cohort_args)
print(TriggerCasePath)

df_case = pd.read_pickle(TriggerCasePath) # (1000)
df_case = df_case.sample(1000, random_state=0).reset_index(drop=True)
ds_case = datasets.Dataset.from_pandas(df_case)
ds_case

{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

Dataset({
    features: ['PID', 'ObsDT'],
    num_rows: 1000
})


## [Step 2] RO: Record Observation & CasePhi


In [4]:
###########################
ROName_List = ['CGM5Min-Bf24H']

name_CasePhi = 'nsSparseTimeIn5MinTS' # non-seq tkn in 5min time step.
###########################

In [5]:
from recfldtkn.obsname import convert_RONameList_to_COName

CaseObsName = convert_RONameList_to_COName(ROName_List, name_CasePhi)
print(CaseObsName)

ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS


In [6]:
# part of fetch_caseobs_Phi_tools
CO_Folder = os.path.join(SPACE['DATA_CaseObs'], CaseObsName)
if not os.path.exists(CO_Folder): os.makedirs(CO_Folder)
print(CO_Folder)

../_Data/2-Data_CaseObs\ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS


# [Part 2] Tools for ds_case_obs

## [Step 1] Prepare examples

In [7]:
case_examples = ds_case[:5]
print(case_examples)

{'PID': [1002840, 1002580, 1004670, 1003564, 1003011], 'ObsDT': [Timestamp('2021-12-07 13:50:00'), Timestamp('2021-11-10 12:15:00'), Timestamp('2021-11-13 06:40:00'), Timestamp('2021-09-15 13:35:00'), Timestamp('2021-10-17 17:15:00')]}


In [8]:
length = len(case_examples[list(case_examples.keys())[0]])
case_examples_list = [{k: v[i] for k, v in case_examples.items()} for i in range(length)]
idx_to_examples = {i: case_examples_list[i] for i in range(length)}
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')},
 1: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')},
 2: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')},
 3: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')},
 4: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}}


In [9]:
case_example = idx_to_examples[0]
case_example

{'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')}

## [Step 2] get_caseobs_id

In [10]:
import inspect
#################################################
def get_CO_id(case_example, CaseObsName, cohort_args):
    RootID = cohort_args['RootID']
    ObsDTName = cohort_args['ObsDTName']
    PIDValue = case_example[RootID]
    ObsDTValue = case_example[ObsDTName].isoformat()
    return f'{PIDValue}&{ObsDTValue}'

get_CO_id.fn_string = inspect.getsource(get_CO_id)
#################################################

In [11]:
caseobs_id = get_CO_id(case_example, CaseObsName, cohort_args)
caseobs_id # CO_id

'1002840&2021-12-07T13:50:00'



## [Step 3] RO List and CasePhi

In [12]:
name_CasePhi 

'nsSparseTimeIn5MinTS'

In [13]:
######### RecObs_Name
print(ROName_List) # RO_List
# Let's focus on one RO
ROName = ROName_List[0]
print(ROName)

['CGM5Min-Bf24H']
CGM5Min-Bf24H


## [Step4] RO Information

In [14]:
# decompose RO to RecName, CkpdName, FldName (small phi)
from recfldtkn.obsname import parse_RecObsName

d = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)
RecName = d['RecName']
CkpdName = d['CkpdName']
FldName = d['FldName']
print(RecName, CkpdName, FldName)

CGM5Min Bf24H None


In [15]:
RecName

'CGM5Min'

In [16]:
# load all information about the R^record_name. 
from recfldtkn.configfn import load_record_args

print(RecName)
rec_args = load_record_args(RecName, cohort_args)
[i for i in rec_args]

CGM5Min


['CohortInfo',
 'RawInfo',
 'RecName',
 'RecID',
 'RecIDChain',
 'DTCols',
 'RecDT',
 'ParentRecName',
 'attr_cols',
 'FldTknInfo',
 'RFT_GROUP_SIZE',
 'idx_group_size',
 'usebucket',
 'GROUP_SIZE',
 'pypath',
 'recfldtkn_config_path',
 'yaml_file_path']

## [Step 5] Load ds_rec

In [17]:
from recfldtkn.loadtools import load_ds_rec_and_info

ds_rec, ds_rec_info = load_ds_rec_and_info(RecName, cohort_args)
# R^Rx
# ROdf # Rx-Bf2M (i, t, Bf2M)

In [18]:
######### column_names (or we can all it attribute columns)
column_names = ds_rec.column_names # attr_columns + phi_columns
column_names
# ROdf_ij --(Phi: RecNum)-> COdf_ij 

['PID',
 'PatientID',
 'CGM5MinID',
 'DT_s',
 'BGValue',
 'CGM5Min-N2Cin1Tkn_wgt',
 'CGM5Min-N2Cin1Tkn_tknidx']

## [Step 6] get_selected_columns (attr + phi) from ds_rec

In [19]:
column_names

['PID',
 'PatientID',
 'CGM5MinID',
 'DT_s',
 'BGValue',
 'CGM5Min-N2Cin1Tkn_wgt',
 'CGM5Min-N2Cin1Tkn_tknidx']

In [20]:
from recfldtkn.obsname import parse_RecObsName
# RecName = RecObs_Name.split('-')[0]
base_columns = [cohort_args['RootID'], rec_args['RecID'], rec_args['RecDT']]
base_columns

['PID', 'CGM5MinID', 'DT_s']

In [21]:
RecName = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)['RecName']
FldName = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)['FldName']
if FldName is None: 
     RecFldTknName = RecName
else:
    RecFldTknName = RecName + '-' + FldName + 'Tkn'
# rec_args['FldTknInfo'][RecFldTknName]
RecFldTknName

'CGM5Min'

In [22]:
###############################
def get_selected_columns(ROName, column_names, cohort_args, rec_args, CaseTkn):
    # from recfldtkn.obsname import parse_RecObsName
    # RecName = parse_RecObsName(ROName)['RecName']
    # FldName = parse_RecObsName(ROName)['FldName']
    base_columns = [cohort_args['RootID'], rec_args['RecID'], rec_args['RecDT']]
    final_cols = base_columns
    # FldName_cols = [i for i in column_names if FldName in i]
    # if len(FldName_cols) == 0: 
    #     parse_RecObsName(ROName)['FldName']
    #     RecFldTknName = RecName + '-' + FldName + 'Tkn'
    #     # rec_args['FldTknInfo'][RecFldTknName]
        
    #     val_cols = rec_args['FldTknInfo'][RecFldTknName]['value_cols']
    #     final_cols = base_columns + val_cols
    # else:
    #     final_cols = base_columns + FldName_cols
    return final_cols

get_selected_columns.fn_string = inspect.getsource(get_selected_columns)
###############################

In [23]:
# big question here
# if small phi is not applied in the record level
# how do we deploy the tools of small phi within this process?
get_selected_columns(ROName, column_names, cohort_args, rec_args, name_CasePhi)

['PID', 'CGM5MinID', 'DT_s']

In [24]:
print(ROName_List)
print(name_CasePhi)
print(get_selected_columns)

['CGM5Min-Bf24H']
nsSparseTimeIn5MinTS
<function get_selected_columns at 0x0000015406D7DBC0>


In [25]:
# RecObsName_to_RecObsInfo # RO_to_ROinfo

from recfldtkn.observer import get_RecObsName_to_RecObsInfo

record_to_ds_rec = {}        # set this to empty dictionary, then we will load data from disk
record_to_ds_rec_info = {}   # set this to empty dictionary, then we will load data from disk
ROName_to_ROInfo = get_RecObsName_to_RecObsInfo(ROName_List, 
                                                name_CasePhi, 
                                                get_selected_columns,
                                                cohort_args, 
                                                cohort_args['ckpd_to_CkpdObsConfig'], 
                                                record_to_ds_rec, 
                                                record_to_ds_rec_info)
    
ROName = ROName_List[0] # Rx-Bf2M  
ROInfo = ROName_to_ROInfo[ROName] # RxInfo + R^Rx (with necessary columns) + Ckpd + FldTkn (phi)

In [26]:
[i for i in ROInfo]
# pprint(ROInfo)

['rec_args',
 'RecName',
 'CkpdName',
 'CkpdInfo',
 'FldName',
 'FldTknName',
 'FldIdx2Tkn',
 'fldtkn_args',
 'fld_tokenizer_fn',
 'run_fldtkn_on_the_fly',
 'ds_rec',
 'df_rec_info']

In [27]:
ROInfo['ds_rec'] # get_selected_columns

Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 21215912
})

## [Step 7] get_CO_vocab

In [28]:
def get_CO_vocab(RecObsName_to_RecObsInfo):
    # phi_list
    RecFldName_list = list(set([RecObsInfo['RecName'] + '-' +RecObsInfo['FldName'] 
                                for RecObsName, RecObsInfo in RecObsName_to_RecObsInfo.items()
                                if RecObsInfo['FldName'] is not None ]
                                ))
    
    # print('RecFldName_list----->', RecFldName_list)
    # for current version: for any Phi, we only process one or zero phi. 
    assert len(RecFldName_list) <= 1

    # Define the range of possible values for each datetime component
    # years = [f'Y{year}' for year in range(2015, 2030)]  # Assuming years 2000-2099
    months = [f'Mon{month}' for month in range(1, 13)]  # 1-12 for months
    weeks_of_year = [f'WOY{week}' for week in range(1, 54)]  # 1-53 for weeks
    weekdays = [f'Weekday{weekday}' for weekday in range(7)]  # 0-6 for weekdays
    hours = [f'Hour{hour}' for hour in range(24)]  # 0-23 for hours
    minutes = [f'Min{minute}' for minute in range(60)]  # 0-59 for minutes
    seconds = [f'Sec{second}' for second in range(60)]  # 0-59 for seconds

    # Combine all possible tokens
    all_tokens = ['None'] + months + weeks_of_year + weekdays + hours + minutes + seconds

    # Generate idx2tkn mapping
    tid2tkn = {index: token for index, token in enumerate(all_tokens)}
    tkn2tid = {token: index for index, token in tid2tkn.items()}

    CaseTknVocab = {}
    CaseTknVocab['tid'] = {'tid2tkn': tid2tkn, 'tkn2tid': tkn2tid}
    CaseTknVocab['wgt'] = {} # we don't use wgt in this version

    RecObsName = [i for i in RecObsName_to_RecObsInfo][0]
    RecObsInfo = RecObsName_to_RecObsInfo[RecObsName]
    CkpdInfo = RecObsInfo['CkpdInfo']
    TimeStepType = '5Min'
    StartIdx = CkpdInfo[f'StartIdx{TimeStepType}']
    EndIdx   = CkpdInfo[f'EndIdx{TimeStepType}']
    idx2tkn = [f'{TimeStepType}_{i}' for i in range(StartIdx, EndIdx + 1)]
    tid2tkn = {tid: tkn for tid, tkn in enumerate(idx2tkn)}
    tkn2tid = {tkn: tid for tid, tkn in tid2tkn.items()}
    CaseTknVocab['timestep'] = {'tid2tkn': tid2tkn, 'tkn2tid': tkn2tid}
    ############################
    
    return CaseTknVocab

get_CO_vocab.fn_string = inspect.getsource(get_CO_vocab)

In [29]:
CO_vocab = get_CO_vocab(ROName_to_ROInfo)
pprint(CO_vocab, sort_dicts=False, compact=True)

{'tid': {'tid2tkn': {0: 'None',
                     1: 'Mon1',
                     2: 'Mon2',
                     3: 'Mon3',
                     4: 'Mon4',
                     5: 'Mon5',
                     6: 'Mon6',
                     7: 'Mon7',
                     8: 'Mon8',
                     9: 'Mon9',
                     10: 'Mon10',
                     11: 'Mon11',
                     12: 'Mon12',
                     13: 'WOY1',
                     14: 'WOY2',
                     15: 'WOY3',
                     16: 'WOY4',
                     17: 'WOY5',
                     18: 'WOY6',
                     19: 'WOY7',
                     20: 'WOY8',
                     21: 'WOY9',
                     22: 'WOY10',
                     23: 'WOY11',
                     24: 'WOY12',
                     25: 'WOY13',
                     26: 'WOY14',
                     27: 'WOY15',
                     28: 'WOY16',
                     29: 'WOY17',
         


## [Step 8] Fetch ds_rec

In [30]:
case_example = idx_to_examples[4]
case_example

{'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}

In [31]:
ROName

'CGM5Min-Bf24H'

In [32]:
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')},
 1: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')},
 2: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')},
 3: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')},
 4: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}}


In [33]:
for ROName, ROInfo in ROName_to_ROInfo.items():
    print(ROName, ':', [i for i in ROInfo])

CGM5Min-Bf24H : ['rec_args', 'RecName', 'CkpdName', 'CkpdInfo', 'FldName', 'FldTknName', 'FldIdx2Tkn', 'fldtkn_args', 'fld_tokenizer_fn', 'run_fldtkn_on_the_fly', 'ds_rec', 'df_rec_info']


In [34]:
# RO_to_ROdf: RecObsName_to_RecObsDS
from recfldtkn.observer import CaseObserverTransformer

get_Record_P = CaseObserverTransformer.get_Record_P
RecName_to_REC_P = get_Record_P(idx_to_examples, ROName_to_ROInfo)
for RecName, REC_P in RecName_to_REC_P.items():
    print(f'============ {RecName} ============')
    pprint(REC_P)

============ CGM5Min ============
{1002580: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 35212
}),
 1002840: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 47367
}),
 1003011: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 51200
}),
 1003564: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 39938
}),
 1004670: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 20889
})}


## [Step 9] Fetch ROdf: $R_i^{recname}$

In [35]:
get_idx_to_RecObsName_to_RecObsDS = CaseObserverTransformer.get_idx_to_RecObsName_to_RecObsDS
idx_to_ROName_to_ROds = get_idx_to_RecObsName_to_RecObsDS(idx_to_examples, ROName_to_ROInfo, RecName_to_REC_P)

for idx, case in idx_to_examples.items():
    print('\n====================')
    print('idx:', idx)
    print('case:', case)
    ROName_to_ROds = idx_to_ROName_to_ROds[idx]
    print('ROName_to_ROds:', ROName_to_ROds) # R_i^{RecNum, Ckpd}
    for ROName, ROds in ROName_to_ROds.items():
        print('ROName:', ROName)
        if ROds is None: 
            print('ROds is None')
            continue
        ROdf = ROds.to_pandas()
        print(ROdf[rec_args['RecDT']])
        # print(ROdf[rec_args['RecID']])  
    print('====================')


idx: 0
case: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')}
ROName_to_ROds: {'CGM5Min-Bf24H': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 289
})}
ROName: CGM5Min-Bf24H
0     2021-12-06 13:50:00
1     2021-12-06 13:55:00
2     2021-12-06 14:00:00
3     2021-12-06 14:05:00
4     2021-12-06 14:10:00
              ...        
284   2021-12-07 13:30:00
285   2021-12-07 13:35:00
286   2021-12-07 13:40:00
287   2021-12-07 13:45:00
288   2021-12-07 13:50:00
Name: DT_s, Length: 289, dtype: datetime64[ns]

idx: 1
case: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')}
ROName_to_ROds: {'CGM5Min-Bf24H': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 289
})}
ROName: CGM5Min-Bf24H
0     2021-11-09 12:15:00
1     2021-11-09 12:20:00
2     2021-11-09 12:25:00
3     2021-11-09 12:30:00
4     2021-11-09 12:35:00
              ...        
284   2021-11-10 11:55:00
285   2021-11-10 12:00:00
286   2021-11-10 12:05:00
287   2021-11-10 12:1

In [36]:
[i for i in ROName_to_ROInfo]

['CGM5Min-Bf24H']

In [37]:
idx = 1
case_example = idx_to_examples[idx]
print('case_example:', case_example)
ROName_to_ROds = idx_to_ROName_to_ROds[idx] # RO_to_ROdf # RO_ij
print('RO_to_ROdf (RO_ij):', ROName_to_ROds)
print([i for i in ROName_to_ROds])

case_example: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')}
RO_to_ROdf (RO_ij): {'CGM5Min-Bf24H': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 289
})}
['CGM5Min-Bf24H']


# [Step 10]: Develop $\Phi$. 

In [38]:
ObsDTValue = case_example['ObsDT'] 
ObsDTValue

Timestamp('2021-11-10 12:15:00')

In [39]:
ROName_to_ROds

{'CGM5Min-Bf24H': Dataset({
     features: ['PID', 'CGM5MinID', 'DT_s'],
     num_rows: 289
 })}

In [40]:
assert len(ROName_to_ROds) == 1 

In [41]:
RecObsName = [i for i in ROName_to_ROds][0] # RO
RecObsName

'CGM5Min-Bf24H'

In [42]:
RecObsDS   = ROName_to_ROds[RecObsName]
RecObsDS

Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s'],
    num_rows: 289
})

In [43]:
RecObsInfo = ROName_to_ROInfo[RecObsName] 
RecObsInfo

{'rec_args': {'CohortInfo': {'RawData2022_CGM': {'ElogBGEntry': {'raw_data_path': '../_Data/0-Data_Raw/RawData2022_CGM/05_02_2022_ElogBGEntry.csv'}},
   'RawData2023_CVSTDCAug': {'ElogBGEntry': {'raw_data_path': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/08_23_2023_ElogBGEntry.csv'}},
   'RawData2023_CVSDeRxAug': {'ElogBGEntry': {'raw_data_path': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/08_23_2023_ElogBGEntry.csv'}}},
  'RawInfo': {'ElogBGEntry': {'RawRootID': 'PatientID',
    'RawName': 'ElogBGEntry',
    'raw_columns': ['PatientID',
     'BGEntryID',
     'ObservationDateTime',
     'EntryCreatedDateTime',
     'ObservationEntryDateTime',
     'ModifiedDateTime',
     'UserObservationDateTime',
     'TimezoneOffset',
     'Timezone',
     'BGValue',
     'ActualBGValue',
     'ExternalSourceID',
     'MeterType']}},
  'RecName': 'CGM5Min',
  'RecID': 'CGM5MinID',
  'RecIDChain': ['PID', 'CGM5MinID'],
  'DTCols': ['DT_s'],
  'RecDT': 'DT_s',
  'ParentRecName': 'P',
  'attr_cols': [

In [44]:
RecDT = RecObsInfo['rec_args']['RecDT'] 
RecDT

'DT_s'

In [45]:
ObsDT = case_example['ObsDT']
ObsDT

Timestamp('2021-11-10 12:15:00')

In [46]:
CkpdInfo = RecObsInfo['CkpdInfo']
CkpdInfo

{'DistStartToPredDT': -24,
 'DistEndToPredDT': 0.01,
 'TimeUnit': 'h',
 'StartIdx5Min': -288,
 'EndIdx5Min': 0}

In [47]:
TimeStepType = '5Min'

StartIdx = CkpdInfo[f'StartIdx{TimeStepType}']
StartIdx

-288

In [48]:
# RO_ds = RecObsDS
# RO_ds

In [49]:
# if RO_ds is not None:
#     df = RO_ds.to_pandas()
# else:
#     df = None
# df

In [50]:
# df[RecDT]

In [51]:
ObsDTValue

Timestamp('2021-11-10 12:15:00')

In [52]:
case_example

{'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')}

In [53]:
# df[RecDT] - ObsDTValue

In [54]:
# df['timestep'] = ((df[RecDT] - ObsDTValue).dt.total_seconds() / (60 * 5)).astype(int)
# df

In [55]:
EndIdx = CkpdInfo[f'EndIdx{TimeStepType}']
timestep_index = range(StartIdx, EndIdx + 1)  # Include 24
timestep_index

range(-288, 1)

In [56]:
StartDateTime = ObsDTValue + pd.to_timedelta(StartIdx* 5, unit='m')
EndDateTime = ObsDTValue + pd.to_timedelta(EndIdx* 5, unit='m')
print(ObsDTValue, StartDateTime, EndDateTime)

2021-11-10 12:15:00 2021-11-09 12:15:00 2021-11-10 12:15:00


In [57]:
CO = {}


TimeStep = 5
TimeUnit = 'munites'
CO['timeinfo'] = ['ObsDTValue', 
                'StartDateTime', 
                'EndDateTime', 
                'TimeStepType', 
                'TimeStepSize',
                'TimeUnit',
                'StartIdx-To-EndIdx', 
                # 'EndIdx'
                ]
CO['timevalues'] = [ObsDTValue.isoformat(), 
                    StartDateTime.isoformat(),
                    EndDateTime.isoformat(), 
                    str(TimeStepType), 
                    str(TimeStep),
                    str(TimeUnit),
                    str(StartIdx) + ':To:' + str(EndIdx), 
                    ]

CO

{'timeinfo': ['ObsDTValue',
  'StartDateTime',
  'EndDateTime',
  'TimeStepType',
  'TimeStepSize',
  'TimeUnit',
  'StartIdx-To-EndIdx'],
 'timevalues': ['2021-11-10T12:15:00',
  '2021-11-09T12:15:00',
  '2021-11-10T12:15:00',
  '5Min',
  '5',
  'munites',
  '-288:To:0']}

In [58]:
import inspect

##################################
def fn_CasePhi(case_example, ROName_to_ROds, ROName_to_ROInfo, CO_vocab, cohort_args):
    # input: RecObsName_to_RecObsDS, RecObsName_to_RecObsInfo
    # output: CaseObservation

    # import itertools 
    # def convert_timestep(x):
    #     x = [i for i in x if type(i) != int]
    #     if len(x) == 0:
    #         return [0] 
    #     else:
    #         # return list(set(list(itertools.chain(*x))))
    #         return list(itertools.chain(*x))
        
    # def extract_datetime_components_as_list(dt):
    #     return [
    #         # f'Y{dt.year}',
    #         f'Mon{dt.month}',
    #         f'WOY{dt.isocalendar().week}',
    #         f'Weekday{dt.weekday()}',
    #         f'Hour{dt.hour}',
    #         f'Min{dt.minute}',
    #         f'Sec{dt.second}'
    #     ]

    ObsDTValue = case_example['ObsDT']   # T_ij value
    assert len(ROName_to_ROds) == 1 # assert that RecObsDS has at least one record   
    RecObsName = [i for i in ROName_to_ROds][0] # RO
    RecObsDS   = ROName_to_ROds[RecObsName]  # RO_ds(RecDT is not caseDT)
    RecObsInfo = ROName_to_ROInfo[RecObsName] # RecObsInfo
    RecDT = RecObsInfo['rec_args']['RecDT'] 
    CkpdInfo = RecObsInfo['CkpdInfo']

    TimeStepType = '5Min'
    StartIdx = CkpdInfo[f'StartIdx{TimeStepType}']
    EndIdx   = CkpdInfo[f'EndIdx{TimeStepType}']
    # timestep_index = range(StartIdx, EndIdx + 1) 


    TimeStep = 5
    TimeUnit = 'minutes'
    StartDateTime = ObsDTValue + pd.to_timedelta(StartIdx* TimeStep, unit=TimeUnit)
    EndDateTime   = ObsDTValue + pd.to_timedelta(EndIdx* TimeStep,   unit=TimeUnit)

    CO = {}
    CO['timeinfo'] = ['ObsDTValue', 
                    # 'StartDateTime', 
                    # 'EndDateTime', 
                    'TimeStepType', 
                    'TimeStepSize',
                    'TimeUnit',
                    'StartIdx-To-EndIdx', 
                    # 'EndIdx'
                    ]
    CO['timevalues'] = [ObsDTValue.isoformat(), 
                        # StartDateTime.isoformat(),
                        # EndDateTime.isoformat(), 
                        str(TimeStepType), 
                        str(TimeStep),
                        str(TimeUnit),
                        str(StartIdx) + ':To:' + str(EndIdx), 
                        ]
    return CO
##################################

fn_CasePhi.fn_string = inspect.getsource(fn_CasePhi)

In [59]:
ROName_to_ROds

{'CGM5Min-Bf24H': Dataset({
     features: ['PID', 'CGM5MinID', 'DT_s'],
     num_rows: 289
 })}

In [60]:
CO = fn_CasePhi(case_example, ROName_to_ROds, ROName_to_ROInfo, CO_vocab, cohort_args)
pprint(CO, compact = True)

{'timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize', 'TimeUnit',
              'StartIdx-To-EndIdx'],
 'timevalues': ['2021-11-10T12:15:00', '5Min', '5', 'minutes', '-288:To:0']}


In [61]:
# CO_vocab

## Save Files

In [62]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [get_CO_id, get_selected_columns, get_CO_vocab, fn_CasePhi]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_casephi', f'phi_{name_CasePhi}.py')
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# Application

In [63]:
CaseObsName

'ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS'

In [64]:
from recfldtkn.observer import get_CaseObsInfo_for_a_CaseObsName

CaseObsInfo = get_CaseObsInfo_for_a_CaseObsName(CaseObsName,
                                                SPACE, 
                                                cohort_args, 
                                                record_to_ds_rec = {}, 
                                                record_to_ds_rec_info = {})
[i for i in CaseObsInfo]

['RecObsName_List',
 'name_CasePhi',
 'get_selected_columns',
 'CaseObsName',
 'ROName_to_ROInfo',
 'fn_CasePhi',
 'get_CO_id',
 'CO_Folder',
 'CO_vocab']

In [65]:
import random 
from pprint import pprint
from datetime import datetime 
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer


ro_to_ROName = {} # leave it empty for now 
ROName_to_ROInfo = CaseObsInfo['ROName_to_ROInfo']
name_CasePhi = CaseObsInfo['name_CasePhi']
fn_CasePhi = CaseObsInfo['fn_CasePhi']
CO_vocab = CaseObsInfo['CO_vocab']
get_CO_id = CaseObsInfo['get_CO_id']
CO_Folder = CaseObsInfo['CO_Folder']
COids = None 
use_CO_from_disk = False
batch_size = CaseObsInfo.get('batch_size', 1000)

fn_caseobs_Phi = CaseObserverTransformer(ro_to_ROName,
                                         ROName_to_ROInfo, 
                                         name_CasePhi, 
                                         fn_CasePhi, 
                                         CO_vocab, 
                                         get_CO_id,
                                         cohort_args,
                                         CO_Folder, 
                                         df_case,
                                         use_CO_from_disk)

# Check Cache Functions

In [66]:
fn_caseobs_Phi.new_COs

{}

In [67]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Elipse Time:  0:00:01.941627
Dataset({
    features: ['PID', 'ObsDT', 'timeinfo', 'timevalues'],
    num_rows: 1000
})


In [68]:
print(len(fn_caseobs_Phi.new_COs))

1000


In [69]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Elipse Time:  0:00:00.018994
Dataset({
    features: ['PID', 'ObsDT', 'timeinfo', 'timevalues'],
    num_rows: 1000
})


In [70]:
print(CaseObsName)
random_int = random.randint(0, len(ds_casetkn))
print(random_int)
pprint(ds_casetkn[random_int])
print(CO_vocab)

ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS
181
{'ObsDT': Timestamp('2021-02-17 00:55:00'),
 'PID': 1000649,
 'timeinfo': ['ObsDTValue',
              'TimeStepType',
              'TimeStepSize',
              'TimeUnit',
              'StartIdx-To-EndIdx'],
 'timevalues': ['2021-02-17T00:55:00', '5Min', '5', 'minutes', '-288:To:0']}
{'tid': {'tid2tkn': {0: 'None', 1: 'Mon1', 2: 'Mon2', 3: 'Mon3', 4: 'Mon4', 5: 'Mon5', 6: 'Mon6', 7: 'Mon7', 8: 'Mon8', 9: 'Mon9', 10: 'Mon10', 11: 'Mon11', 12: 'Mon12', 13: 'WOY1', 14: 'WOY2', 15: 'WOY3', 16: 'WOY4', 17: 'WOY5', 18: 'WOY6', 19: 'WOY7', 20: 'WOY8', 21: 'WOY9', 22: 'WOY10', 23: 'WOY11', 24: 'WOY12', 25: 'WOY13', 26: 'WOY14', 27: 'WOY15', 28: 'WOY16', 29: 'WOY17', 30: 'WOY18', 31: 'WOY19', 32: 'WOY20', 33: 'WOY21', 34: 'WOY22', 35: 'WOY23', 36: 'WOY24', 37: 'WOY25', 38: 'WOY26', 39: 'WOY27', 40: 'WOY28', 41: 'WOY29', 42: 'WOY30', 43: 'WOY31', 44: 'WOY32', 45: 'WOY33', 46: 'WOY34', 47: 'WOY35', 48: 'WOY36', 49: 'WOY37', 50: 'WOY38', 51: 'WOY39'

In [71]:
ds_casetkn[4]

{'PID': 1003011,
 'ObsDT': Timestamp('2021-10-17 17:15:00'),
 'timeinfo': ['ObsDTValue',
  'TimeStepType',
  'TimeStepSize',
  'TimeUnit',
  'StartIdx-To-EndIdx'],
 'timevalues': ['2021-10-17T17:15:00', '5Min', '5', 'minutes', '-288:To:0']}

## Save

In [72]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [73]:
CO_Folder_data = fn_caseobs_Phi.CO_Folder_data 
CO_Folder_data

'../_Data/2-Data_CaseObs\\ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS\\data'

In [74]:
fn_caseobs_Phi.save_new_COs_to_disk(CO_Folder_data)

In [75]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [76]:
fn_caseobs_Phi.ds_CO_data

# Save Vocab

In [77]:
CO_vocab = fn_caseobs_Phi.CO_vocab
CO_vocab

{'tid': {'tid2tkn': {0: 'None',
   1: 'Mon1',
   2: 'Mon2',
   3: 'Mon3',
   4: 'Mon4',
   5: 'Mon5',
   6: 'Mon6',
   7: 'Mon7',
   8: 'Mon8',
   9: 'Mon9',
   10: 'Mon10',
   11: 'Mon11',
   12: 'Mon12',
   13: 'WOY1',
   14: 'WOY2',
   15: 'WOY3',
   16: 'WOY4',
   17: 'WOY5',
   18: 'WOY6',
   19: 'WOY7',
   20: 'WOY8',
   21: 'WOY9',
   22: 'WOY10',
   23: 'WOY11',
   24: 'WOY12',
   25: 'WOY13',
   26: 'WOY14',
   27: 'WOY15',
   28: 'WOY16',
   29: 'WOY17',
   30: 'WOY18',
   31: 'WOY19',
   32: 'WOY20',
   33: 'WOY21',
   34: 'WOY22',
   35: 'WOY23',
   36: 'WOY24',
   37: 'WOY25',
   38: 'WOY26',
   39: 'WOY27',
   40: 'WOY28',
   41: 'WOY29',
   42: 'WOY30',
   43: 'WOY31',
   44: 'WOY32',
   45: 'WOY33',
   46: 'WOY34',
   47: 'WOY35',
   48: 'WOY36',
   49: 'WOY37',
   50: 'WOY38',
   51: 'WOY39',
   52: 'WOY40',
   53: 'WOY41',
   54: 'WOY42',
   55: 'WOY43',
   56: 'WOY44',
   57: 'WOY45',
   58: 'WOY46',
   59: 'WOY47',
   60: 'WOY48',
   61: 'WOY49',
   62: 'WOY50',
   

In [78]:
CO_Folder_vocab = fn_caseobs_Phi.CO_Folder_vocab
CO_Folder_vocab

'../_Data/2-Data_CaseObs\\ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS\\vocab.p'

In [79]:
df_Vocab = pd.DataFrame({CaseObsName: CO_vocab})
df_Vocab.to_pickle(CO_Folder_vocab)
df_Vocab

,ro.CGM5Min-Bf24H_ct.nsSparseTimeIn5MinTS
tid,"{'tid2tkn': {0: 'None', 1: 'Mon1', 2: 'Mon2', ..."
wgt,{}
timestep,"{'tid2tkn': {0: '5Min_-288', 1: '5Min_-287', 2..."
